## Acknowledgments/Authors

This notebook was developed collaboratively by Umesh Rawat and Ari Tal. Both contributed to various parts of the content and code.

In [1]:
!pip install fastapi uvicorn nest-asyncio
!pip install openai
!pip install --upgrade pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.8 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import json
from openai import OpenAI
import requests


In [3]:
# Tunnel agent api key found here: https://dashboard.ngrok.com/tunnels/authtokens
ngrok.set_auth_token("") #Ari's token

In [4]:
app = FastAPI()

@app.post("/getNearbyConsumables")
async def root(data: dict):
  address = data.get("address")
  if address is None:
    return {"error": "Address not provided"}
  else:
    results = process_data(address)
    print(results)
  return {"result": results}

In [14]:




def get_ai_snippets_for_query(query):
    # api key found here: https://api.you.com/api-key
    my_api_key = ''
    headers = {"X-API-Key": my_api_key}
    params = {"query": query}
    return requests.get(
        f"https://api.ydc-index.io/search",
        params=params,
        headers=headers,
    ).json()

def do_queries(location="565 Market St. San Francisco, CA 94105"):
    '''
        Input: location (string)
        Description: Does 3 web searches: one for nearby restaurants, one for nearby bookstores,
        and one for nearby fun places.
        Returns the results of each search.

        Output: results_restaurants, results_bookstores, results_places
    '''
    temp_prompt_restaurants = f"Find restaurants near {location}"
    results_restaurants = get_ai_snippets_for_query(temp_prompt_restaurants)

    temp_prompt_bookstores = f"Find bookstores near {location}"
    results_bookstores = get_ai_snippets_for_query(temp_prompt_bookstores)

    temp_prompt_places = f"Find fun places near {location}"
    results_places = get_ai_snippets_for_query(temp_prompt_places)
    return results_restaurants, results_bookstores, results_places


def get_nearby_places(data_restaurants, data_bookstores, data_other):
    # Open AI API key can be found here: https://platform.openai.com/api-keys
    client = OpenAI(
        api_key="",  # this is also the default, it can be omitted
    )
    # final_prompt = f"Provide a list of nearby restaurants, bookstores, and other interesting places to visit near {location} in JSON format. Format the response as a JSON object with keys 'restaurants', 'bookstores', and 'places'. Don't worry about providing real-time information. Just tell me what you know. Also, please make your entire response in json format without any context."
    final_prompt = f"I did a web search to find nearby retaurants. It is in a wierd format: ```\n{data_restaurants}\n```\n\n" + \
                   f"I did a web search to find nearby bookstores. It is in a wierd format: ```\n{data_bookstores}\n```\n\n" + \
                   f"I did a web search to find nearby fun other places. It is in a wierd format: ```\n{data_other}\n```\n\n" + \
                   f"Please use this information to provide a JSON object listing nearby restaurants, bookstores, and other interesting places " + \
                   f"to visit. The object should contain three keys: 'restaurants', 'bookstores', " + \
                   f"and 'other_places'. Populate each key with a list of examples based on your knowledge. " + \
                   f"Each example should include a name, an address, a url, and a description. Rewrite the description to be fanciful.\n\n" + \
                   f"Do not include any additional text or explanations—just return the JSON object.\n\n" + \
                   f"To clarify, please do not include the word 'json' before the json object. I really do JUST want the json object."
    for _ in range(5):
        response = client.chat.completions.create(
                  model="gpt-4o",
                  messages=[
                      {"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": final_prompt}
                ]
              )
        content = response.choices[0].message.content
        try:
            nearby_places = json.loads(content)
            return nearby_places
        except json.JSONDecodeError:
            print("Response content is not valid JSON.")
            continue



In [17]:
def process_data(value):
  # prompt = f"I am at : {value}. Please provide me the best restaurants and clothing store around me."
  # print(prompt)

  results_restaurants, results_bookstores, results_places = do_queries(location=value)
  nearby_places = get_nearby_places(results_restaurants['hits'][:10], results_bookstores['hits'][:10], results_places['hits'][:10])

  return nearby_places # example processing: double the value

In [ ]:
ngrok_tunnel = ngrok.connect(8095)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8095)

INFO:     Started server process [1224]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8095 (Press CTRL+C to quit)


Public URL: https://739f-35-237-197-153.ngrok-free.app
INFO:     3.88.217.234:0 - "GET / HTTP/1.1" 404 Not Found
{'restaurants': [{'name': 'Prospect', 'address': '300 Spear St, San Francisco, CA 94105', 'url': 'https://www.opentable.com/prospect', 'description': 'Prospect is a casual elegant American eatery where modern cuisine meets relaxed sophistication. Book a table to savor delectable dishes amidst chic surroundings.'}], 'bookstores': [{'name': 'City Lights Booksellers & Publishers', 'address': '261 Columbus Ave, San Francisco, CA 94133', 'url': 'https://www.citylights.com/', 'description': 'City Lights is a historic literary haven where book lovers can lose themselves among endless shelves of fascinating reads and immerse in the spirit of literary enlightenment.'}, {'name': 'Green Apple Books', 'address': '506 Clement St, San Francisco, CA 94118', 'url': 'https://www.greenapplebooks.com/', 'description': 'Green Apple Books is a charming and eclectic bookstore offering a plethora 